In [ ]:
from google.colab import files
files.upload()  # Upload the kaggle.json file here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mirdul2403","key":"1b21ecb3f99aedbd11d6d1009402d42f"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images


Dataset URL: https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images
License(s): other
 78% 82.0M/105M [00:00<00:00, 855MB/s]
100% 105M/105M [00:00<00:00, 872MB/s] 


In [ ]:
!unzip -q cifake-real-and-ai-generated-synthetic-images.zip -d data

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),   # Resize images to 224x224
    transforms.ToTensor(),           # Convert to Tensor
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])  # Normalize
])

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])


In [ ]:
full_train_dataset = datasets.ImageFolder('data/train', transform=transform)


In [ ]:
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size

train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])


In [ ]:
test_dataset = datasets.ImageFolder('data/test', transform=transform)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Train images:", len(train_dataset))
print("Validation images:", len(val_dataset))
print("Test images:", len(test_dataset))


Train images: 80000
Validation images: 20000
Test images: 20000


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm  # For pre-trained models


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cuda


In [ ]:
model = timm.create_model('resnet18', pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes
model = model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
epochs = 5  # You can increase later
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {100*correct/total:.2f}%")


Epoch 1/5 | Loss: 0.1204 | Val Accuracy: 96.52%
Epoch 2/5 | Loss: 0.0841 | Val Accuracy: 96.91%
Epoch 3/5 | Loss: 0.0600 | Val Accuracy: 97.23%
Epoch 4/5 | Loss: 0.0428 | Val Accuracy: 97.43%
Epoch 5/5 | Loss: 0.0317 | Val Accuracy: 97.48%


In [ ]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")


Test Accuracy: 97.42%


In [ ]:
torch.save(model.state_dict(), 'ai_image_detector.pth')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
!ls


/content
ai_image_detector.pth				   data   kaggle.json
cifake-real-and-ai-generated-synthetic-images.zip  drive  sample_data


In [ ]:
!cp /content/ai_image_detector.pth /content/drive/MyDrive/


LOAD MODEL >>>

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image


In [ ]:

model = timm.create_model('resnet18', pretrained=False) # Changed from resnet50 to resnet18
model.fc = torch.nn.Linear(model.fc.in_features, 2)  # 2 classes: real / ai

# Load the trained weights
model.load_state_dict(torch.load('/content/drive/MyDrive/ai_image_detector.pth', map_location='cpu'))
model.eval()

print("✅ Model loaded successfully!")

✅ Model loaded successfully!


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Example: your uploaded test image
image_path = '/content/drive/MyDrive/data/NewTestDataIBM/download.jpg'  # change this path

img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0)  # shape: [1, 3, 224, 224]


In [ ]:
# Class labels — update if yours are different
classes = ['real', 'ai_generated']

# Ensure device is defined and consistent
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) # Ensure the model is on the correct device

# Move the input tensor to the same device as the model
input_tensor = input_tensor.to(device)

with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)

print(f"Predicted raw value: {predicted.item()}") # Added for debugging

# Check if the predicted value is within the range of class indices
if 0 <= predicted.item() < len(classes):
    print(f"Prediction: {classes[predicted.item()]}")
else:
    print(f"Error: Predicted value {predicted.item()} is out of range for classes with indices 0 to {len(classes)-1}.")

Predicted raw value: 0
Prediction: real


ResNET

In [ ]:
import torch
from torchvision import datasets, transforms
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
test_data = datasets.ImageFolder('/content/data/test', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
# Assuming 'timm' is imported (as in cell 6C3dvpR03Pjj)
import timm
import torch

# Use timm.create_model to match the training setup
model = timm.create_model('resnet18', pretrained=False)
# Modify the final fully connected layer for 2 output classes
model.fc = torch.nn.Linear(model.fc.in_features, 2)

# Load the trained weights
model.load_state_dict(torch.load('/content/drive/MyDrive/ai_image_detector.pth', map_location='cpu'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [ ]:
import torch
from tqdm.notebook import tqdm  # shows a nice progress bar

y_true = []
y_pred = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval()

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating", unit="batch"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print("✅ Evaluation complete!")


Evaluating:   0%|          | 0/625 [00:00<?, ?batch/s]

✅ Evaluation complete!


In [ ]:
print(classification_report(y_true, y_pred, target_names=['real', 'ai_generated']))


              precision    recall  f1-score   support

        real       0.68      0.99      0.81     10000
ai_generated       0.98      0.54      0.70     10000

    accuracy                           0.77     20000
   macro avg       0.83      0.77      0.75     20000
weighted avg       0.83      0.77      0.75     20000



In [ ]:
from PIL import Image # Import the Image class

img_path = '/content/1_LqClB-Wa__W9WrNqP74I3Q.png'  # replace with your uploaded file
img = Image.open(img_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0)

# Assuming 'model' and 'transform' are already defined and available
# Ensure device is defined and consistent
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) # Ensure the model is on the correct device
input_tensor = input_tensor.to(device) # Move the input tensor to the same device as the model

# Class labels - update if yours are different
classes = ['real', 'ai_generated']


with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)

print(f"Prediction: {classes[predicted.item()]}")

Prediction: real


BitMind + Trained Model

In [ ]:
%%writefile hybrid_ai_detector.py

Writing hybrid_ai_detector.py


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import requests
import io


In [ ]:
BITMIND_URL = "https://api.bitmind.ai/oracle/v1/34/detect-image"
BITMIND_KEY = "oracle-dc220854-d6f1-4282-92d6-55d4f9fac521:64b25953"  # your given token
MODEL_PATH = "/content/drive/MyDrive/ai_image_detector.pth"

In [ ]:
W_API = 0.5   # BitMind weight
W_MODEL = 0.5  # Your model weight
THRESHOLD = 0.5

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import requests
import io
import timm # Import timm for model creation

class AIImageDetector(nn.Module):
    def __init__(self):
        super(AIImageDetector, self).__init__()
        # Instantiate the model architecture
        self.model = timm.create_model('resnet18', pretrained=False)
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, 2) # Modify for 2 output classes

        # Load the state dictionary into the model
        state_dict = torch.load(MODEL_PATH, map_location=device)
        self.model.load_state_dict(state_dict)

        self.model.eval() # Now it's safe to call eval() on the model object

    def predict(self, image):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        img_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = self.model(img_tensor)
            # Assuming the output is logits, apply sigmoid to get a probability for one of the classes
            # For binary classification with CrossEntropyLoss during training, the output is typically logits.
            # Sigmoid converts logits to a probability between 0 and 1.
            # Let's assume the probability of being 'ai_generated' is the second element (index 1) after softmax, or apply sigmoid to the output of the AI class logit
            # A common approach for binary classification with CrossEntropyLoss is to interpret the output directly as logits for two classes.
            # To get a single probability of being AI-generated, you can use softmax and take the probability of the 'ai_generated' class (assuming it's index 1).
            # Or, if your model's final layer was a single output for the probability of being AI, you'd use sigmoid.
            # Given the training with CrossEntropyLoss and 2 output features, the output is likely logits.
            # We can take the logit for the 'ai_generated' class (assuming index 1) and apply sigmoid to get a probability.
            prob = torch.sigmoid(output[:, 1]).item() # Get the logit for the second class and apply sigmoid

        return prob

model = AIImageDetector()

In [ ]:
def call_bitmind_api(image_bytes):
    headers = {"Authorization": f"Bearer {BITMIND_KEY}"}
    # Use a more specific filename and content type
    files = {"image": ("image.jpg", image_bytes, "image/jpeg")}
    resp = requests.post(BITMIND_URL, headers=headers, files=files)

    # Check for HTTP errors and print response content for debugging
    try:
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print("API Response Content:")
        print(resp.text) # Print the response body
        raise # Re-raise the exception after printing

    data = resp.json()
    # Be more explicit about accessing the score/confidence
    score_api = data.get("confidence", data.get("score", 0.5)) # Use get with a default
    return float(score_api), data

In [ ]:
def get_weighted_result(image_path):
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

    # Get model and API confidences
    score_model = model.predict(image)
    score_api, api_resp = call_bitmind_api(image_bytes)

    # Weighted mean
    weighted_mean = ((W_API * score_api) + (W_MODEL * score_model)) / (W_API + W_MODEL)

    # Final decision
    prediction = "AI-generated" if weighted_mean >= THRESHOLD else "Real"

    return {
        "score_api": round(score_api, 4),
        "score_model": round(score_model, 4),
        "weighted_mean": round(weighted_mean, 4),
        "result": prediction
    }

In [ ]:
if __name__ == "__main__":
    test_image = "/content/premium_photo-1664474619075-644dd191935f.jpg"  # raw string
    result = get_weighted_result(str(test_image))
    print(result)

{'score_api': 1.0, 'score_model': 0.0, 'weighted_mean': 0.5, 'result': 'AI-generated'}


In [ ]:
from google.colab import files
files.download("hybrid_ai_detector.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
torch.save(model, "/content/drive/MyDrive/hybrid_bitmind_model.pth")

print("✅ Hybrid model saved as /content/drive/MyDrive/hybrid_bitmind_model.pth")

✅ Hybrid model saved as /content/drive/MyDrive/hybrid_bitmind_model.pth
